# Om en massa history data saknas eller är felaktig
- ## hitta alla saknade hist / alt läs in förberedd fil med felaktigheter
- ## Scrape på dem och spara i nya_hist.csv
# Kör "Starta här!!!" nedan 


In [1]:
#!apt install chromium-chromedriver
!pip install selenium
#!pip install tabulate
#!pip install catboost

In [2]:
import pandas as  pd 
import numpy as np
import requests
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from tabulate import tabulate

import urllib.request
#from bs4 import BeautifulSoup
import time
import logging
logging.basicConfig(filename='hist.log', filemode='w', format='%(name)s - %(message)s',level=logging.INFO)


In [3]:
def kom_till_start(the_driver,avd,i):
    
    pth='//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div['+avd+']/div/div/table/tbody/tr['+str(i)+']/td[1]'
    try:
        kom = the_driver.find_element_by_xpath(pth)
    except:
        return False

    if kom.text == 's':
        return False
    if kom.text == 'utg':
        return False

    return kom.text

def getNummer(the_driver,avd,i):
       
    pth='//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div['+avd+']/div/div/table/tbody/tr['+str(i)+']/td[2]/span/span'
    
    nummer = the_driver.find_element_by_xpath(pth) 
    return nummer.text

def getBana(the_driver,avd):
    div = '2' if avd == '7' else '1' ## konstigt konstigt!

    pth='//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div['+avd+']/div/div/div['+div+']/div[1]/div[1]/div/div[2]/div[2]/div[1]/span'
    
    bana = the_driver.find_element_by_xpath(pth) 
    
    return bana.text
 



## Selektera ut missing history ur mer_komplett

In [4]:
#### Använd denna för missing hist ####
def missing_Hist():
    df=pd.read_csv('mer_komplett.csv')
    df=df[df.h1_dat.isna()]
    df=df.sort_values(by=['datum','bana','avd','häst'])
    print('storlek "missing hist"',df.shape)
    return df

In [5]:
#### Använd denna för att rätta felaktig hist-data
def felaktig_Hist():
    df=pd.read_csv('felaktigHist.csv')
    print('storlek "felaktig hist"',df.shape)
    return df


In [6]:

url =  'https://www.atg.se/spel/2015-01-03/V75/jagersro/'

driver_s = webdriver.Chrome(executable_path='C:\\Users\peter\\Documents\\MyProjects\\gecko\\chromedriver.exe')

driver_s.get(url)              # startlist
driver_s.implicitly_wait(5)     # seconds
driver_s.fullscreen_window()

input('MARKERA INGET ')




''

# Starta Här!
## Loop through the rows with missing hist

In [7]:
def get_one_hist(the_driver, avd,nr,child):
    one_child=[]
    da='1'  # datum
    ba='2'  # bana
    ku='3'  # kusk
    pl='5'  # plac
    di='6'  # dist
    sp='7'  # spår
    km='10' # km-tid
    od='12' # odds
    pr='13' # pris
    part1='//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div['+avd+']/div/div/table/tbody/tr['+nr+']/td/div[2]/table/tbody/'
    try:
        datum = part1+'tr['+str(child)+']/td['+da+']' #/span/a/div/span'
        datum = the_driver.find_element_by_xpath(datum)
        print('datum',datum.text)
    except:
        print("##### hist except på datum ######")
        #     häst_rad.click()
        #     datum = driver_s.find_element_by_xpath(datum)
        return False
            
    bana =  part1 + 'tr['+str(child)+']/td['+ba+']'
    kusk =  part1 + 'tr['+str(child)+']/td['+ku+']'
    plac =  part1 + 'tr['+str(child)+']/td['+pl+']'
    dist =  part1 + 'tr['+str(child)+']/td['+di+']'
    spår =  part1 + 'tr['+str(child)+']/td['+sp+']'
    KMtid = part1 + 'tr['+str(child)+']/td['+km+']'
    odds =  part1 + 'tr['+str(child)+']/td['+od+']'
    pris =  part1 + 'tr['+str(child)+']/td['+pr+']'
    bana = the_driver.find_element_by_xpath(bana).text
    bana = bana.split('-')[0]
    #print('bana',bana)

    kusk = the_driver.find_element_by_xpath(kusk)
    #print('kusk', kusk.text)

    plac = the_driver.find_element_by_xpath(plac)
    #print('plac',plac.text)

    dist = the_driver.find_element_by_xpath(dist)
    #print('dist',dist.text)
    
    spår = the_driver.find_element_by_xpath(spår)
    #print('spår',spår.text)
    
    KMtid = the_driver.find_element_by_xpath(KMtid)
    #print('kmtid',KMtid.text)
    
    odds = the_driver.find_element_by_xpath(odds)
    #print('odds',odds.text)
    
    pris = the_driver.find_element_by_xpath(pris)
    #print('pris',pris.text)

    one_child = [datum.text, bana,kusk.text, plac.text, dist.text, spår.text, KMtid.text, odds.text, pris.text]

    return one_child


In [8]:
def save_allHist(allHist):
    dfHist=pd.DataFrame(allHist,columns=['datum','bana','avd','häst',  
    'h1_dat', 'h1_bana', 'h1_kusk', 'h1_plac', 'h1_dist','h1_spår','h1_kmtid','h1_odds','h1_pris',
    'h2_dat', 'h2_bana', 'h2_kusk', 'h2_plac', 'h2_dist','h2_spår','h2_kmtid','h2_odds','h2_pris',
    'h3_dat', 'h3_bana', 'h3_kusk', 'h3_plac', 'h3_dist','h3_spår','h3_kmtid','h3_odds','h3_pris',
    'h4_dat', 'h4_bana', 'h4_kusk', 'h4_plac', 'h4_dist','h4_spår','h4_kmtid','h4_odds','h4_pris',
    'h5_dat', 'h5_bana', 'h5_kusk', 'h5_plac', 'h5_dist','h5_spår','h5_kmtid','h5_odds','h5_pris'
    ])
    dfHist.to_csv('dfHist.csv',index=False)

In [9]:
#### df=missing_Hist()   alternativ ####
df=felaktig_Hist()  #### alternativ
df.sort_values(by=['datum','avd'],ascending=[False,True],  inplace=True)
df.reset_index(drop=True,inplace=True)

allHist=[]

#### endast för återstart==True
återstart = False
datumStart='2016-03-05'
avdStart=5
#### 
    
spara_dat = ' '
cnt=0
for datum,bana,avd,häst in df[['datum','bana','avd','häst']].values:
    #### Denna loop för att fortsätta från ett fel-avbrott ####
    if återstart:
        if datum != datumStart:
            continue        
        if avd != avdStart:
            continue
        återstart=False
        logging.warning(f'Nu kör vi igen från {datum} {bana} avd={avd}')
    #### Nu fortsätter vi från senast

    if spara_dat != datum:
        url =  'https://www.atg.se/spel/'+datum+'/V75/'
        driver_s.get(url)              # startlist
        driver_s.implicitly_wait(5)     # seconds
        driver_s.fullscreen_window()
    spara_dat=datum    
    print(datum, bana, avd, häst)
    
    # peka ut rätt avd
    avd_pth = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div['+str(avd)+']/div/div/table/tbody'
    avd_tot = driver_s.find_elements_by_xpath(avd_pth)

    # lista med alla hästrader inom detta lopp
    lista = avd_tot[0].text.split('\n') 
    try:
        nr = [i for i, x in enumerate(lista) if häst in x][0]
    except:
        print(f'{häst} saknas i lista. {datum} {bana} avd={avd}')  
        logging.warning(f'{häst} saknas i lista. {datum} {bana} avd={avd}')         
        continue

    # peka ut och klicka på rätt häst-rad
    h = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div['+str(avd)+']/div/div/table/tbody/tr['+str(nr+1)+']'
    horse = driver_s.find_element_by_xpath(h)
    driver_s.implicitly_wait(2)     # seconds
        
    horse.click()
    driver_s.implicitly_wait(2)     # seconds
        
    # Hästens historik,
    history = []
    for child in range(1,6):
        one_hist = get_one_hist(driver_s,str(avd), str(nr+2), str(child))
        if one_hist:
            history += one_hist
        else:
            logging.warning(f'hist missing {datum} {bana} avd={avd} häst={häst}')
            print(f'hist missing {datum} {bana} avd={avd} häst={häst}')
            history += [None, None,None, None, None, None, None, None, None]
            continue

    horse.click()
    
    allHist += [[datum,bana,avd,häst]+history]
    cnt += 1
    if cnt%100==0:
        save_allHist(allHist)

dfHist=pd.DataFrame(allHist,columns=['datum','bana','avd','häst',  
    'h1_dat', 'h1_bana', 'h1_kusk', 'h1_plac', 'h1_dist','h1_spår','h1_kmtid','h1_odds','h1_pris',
    'h2_dat', 'h2_bana', 'h2_kusk', 'h2_plac', 'h2_dist','h2_spår','h2_kmtid','h2_odds','h2_pris',
    'h3_dat', 'h3_bana', 'h3_kusk', 'h3_plac', 'h3_dist','h3_spår','h3_kmtid','h3_odds','h3_pris',
    'h4_dat', 'h4_bana', 'h4_kusk', 'h4_plac', 'h4_dist','h4_spår','h4_kmtid','h4_odds','h4_pris',
    'h5_dat', 'h5_bana', 'h5_kusk', 'h5_plac', 'h5_dist','h5_spår','h5_kmtid','h5_odds','h5_pris'
    ])
dfHist

8 00:00:00 HALMSTAD avd=6
2015-02-28 00:00:00 HALMSTAD 6 YOUNG FARMER
YOUNG FARMER saknas i lista. 2015-02-28 00:00:00 HALMSTAD avd=6
2015-02-28 00:00:00 HALMSTAD 7 AROMA GRIM
AROMA GRIM saknas i lista. 2015-02-28 00:00:00 HALMSTAD avd=7
2015-02-28 00:00:00 HALMSTAD 7 ARTEMIS
ARTEMIS saknas i lista. 2015-02-28 00:00:00 HALMSTAD avd=7
2015-02-28 00:00:00 HALMSTAD 7 DET ÄR K.H.K. BOKO
DET ÄR K.H.K. BOKO saknas i lista. 2015-02-28 00:00:00 HALMSTAD avd=7
2015-02-28 00:00:00 HALMSTAD 7 GEISHA SUND
GEISHA SUND saknas i lista. 2015-02-28 00:00:00 HALMSTAD avd=7
2015-02-28 00:00:00 HALMSTAD 7 J.S.ANNABELL
J.S.ANNABELL saknas i lista. 2015-02-28 00:00:00 HALMSTAD avd=7
2015-02-28 00:00:00 HALMSTAD 7 LA PRINSESS
LA PRINSESS saknas i lista. 2015-02-28 00:00:00 HALMSTAD avd=7
2015-02-28 00:00:00 HALMSTAD 7 RADYSIN AMERICA
RADYSIN AMERICA saknas i lista. 2015-02-28 00:00:00 HALMSTAD avd=7
2015-02-28 00:00:00 HALMSTAD 7 RINA CHEN HAI
RINA CHEN HAI saknas i lista. 2015-02-28 00:00:00 HALMSTAD avd=7


,datum,bana,avd,häst,h1_dat,h1_bana,h1_kusk,h1_plac,h1_dist,h1_spår,...,h4_pris,h5_dat,h5_bana,h5_kusk,h5_plac,h5_dist,h5_spår,h5_kmtid,h5_odds,h5_pris
0,2021-03-06 00:00:00,ÅBY,1,MR PERFECTION,210306,Åby,Jim Oscarsson,3,2140,2,...,250',210126,Axevalla,Kevin Oscarsson,1,2140,3,"14,3a","4,22",35'
1,2021-02-20 00:00:00,MANTORP,4,APPALOOSA,210220,Mantorp,Bo Falsig,3,1640,5,...,110',201205,Åby,Bo Falsig,7,2140,15,"14,3a","42,78",110'
2,2021-02-13 00:00:00,ÅBY,3,TIMO LANE,210305,Solvalla,Robert Bergh,1,2140,2,...,337',201212,Bergsåker,Rikard N Skoglund,3,3140,1,"16,5a","11,16",110'
3,2021-02-06 00:00:00,SOLVALLA,4,ALHAMBRA MAIL,210206,Solvalla,Carl Johan Jepson,1,2160,4,...,110',201125,Åby,Carl Johan Jepson,2,2640,14,"14,9a","6,98",60'
4,2021-02-06 00:00:00,SOLVALLA,4,LYX HÅLERYD,210227,Romme,Rikard N Skoglund,4,2660,4,...,110',210102,Kalmar,Rikard N Skoglund,5,2160,7,"14,4","6,77",110'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,2016-05-07 00:00:00,ÖREBRO,1,VALEUR,210305,Solvalla,Sandra Nordemo,3,2140,2,...,30',201014,Solvalla,Sandra Nordemo,5,1640,1,"11,3a","102,45",60'
136,2016-04-30 00:00:00,ÅBY,1,HANK VON HÅLERYD,210101,Örebro,Jan Henriksson,d,2160,1,...,30',190309,Cagnes sur Mer,Jean Charles Feron,7,1609,,"12,0a","17,00",
137,2016-01-31 00:00:00,SOLVALLA,1,COUNTERFEIGHTR,210228,Halmstad,Kevin Oscarsson,2,2140,1,...,50',201114,Romme,Per Lennartsson,3,2140,5,"12,9a","37,76",110'
138,2015-12-19 00:00:00,MANTORP,6,THENGILL,210302,Axevalla,Carl Johan Jepson,3,2140,2,...,30',200503,Vaggeryd,Peter Untersteiner,1,2680,1,"14,8","4,83",35'


In [ ]:

dfHist.info()

## När allt är klart - spara resultat och quit web-driver

In [10]:
dfHist.shape

(140, 49)

In [ ]:
# dfHist = pd.concat([pd.read_csv('dfHist1.csv'),
#                     pd.read_csv('dfHist.csv')])
# dfHist=pd.read_csv('dfHist.csv')
# dfHist.drop('Unnamed: 0',axis=1,inplace=True)                    
print(dfHist.shape)
#ta bort dubletter
dfHist.drop_duplicates(['datum','häst'],inplace=True)
dfHist.sort_values(by=['datum','bana','avd'],inplace=True)
dfHist.reset_index(drop=True,inplace=True)
dfHist.shape

In [ ]:
dfHist.shape
dfHist.to_csv('mer_history.csv', index=False)

In [11]:
driver_s.quit()

## lite av varje

In [ ]:
dfHist.info()

In [ ]:
dfHist.h1_dat=dfHist.h1_dat.str.replace('(','',regex=False)
dfHist.h2_dat=dfHist.h2_dat.str.replace('(','',regex=False)
dfHist.h3_dat=dfHist.h3_dat.str.replace('(','',regex=False)
dfHist.h4_dat=dfHist.h4_dat.str.replace('(','',regex=False)
dfHist.h5_dat=dfHist.h5_dat.str.replace('(','',regex=False)
dfHist.h1_dat=dfHist.h1_dat.str.replace(')','',regex=False)
dfHist.h2_dat=dfHist.h2_dat.str.replace(')','',regex=False)
dfHist.h3_dat=dfHist.h3_dat.str.replace(')','',regex=False)
dfHist.h4_dat=dfHist.h4_dat.str.replace(')','',regex=False)
dfHist.h5_dat=dfHist.h5_dat.str.replace(')','',regex=False)
dfHist.h1_dat = dfHist.h1_dat.astype(int)
dfHist.h2_dat = dfHist.h2_dat.astype(int)
dfHist.h3_dat = dfHist.h3_dat.astype(int)
dfHist.h4_dat = dfHist.h4_dat.astype(int)
dfHist.h5_dat = dfHist.h5_dat.astype(int)



In [ ]:
# dfHist.h1_pris=dfHist.h1_pris.str.replace(' ','',regex=False)
dfHist.h1_pris.unique()

In [ ]:
dfHist.h1_pris = dfHist.h1_pris.str.replace("'","")
dfHist.h1_pris[dfHist.h1_pris==""] = np.nan
dfHist.h2_pris = dfHist.h2_pris.str.replace("'","")
dfHist.h2_pris[dfHist.h2_pris=='']=np.nan
dfHist.h3_pris = dfHist.h3_pris.str.replace("'","")
dfHist.h3_pris[dfHist.h3_pris=='']=np.nan
dfHist.h4_pris = dfHist.h4_pris.str.replace("'","")
dfHist.h4_pris[dfHist.h4_pris=='']=np.nan
dfHist.h5_pris = dfHist.h5_pris.str.replace("'","")
dfHist.h5_pris[dfHist.h5_pris=='']=np.nan
dfHist.h1_pris = dfHist.h1_pris.astype('float')
dfHist.h2_pris = dfHist.h2_pris.astype('float')
dfHist.h3_pris = dfHist.h3_pris.astype('float')
dfHist.h4_pris = dfHist.h4_pris.astype('float')
dfHist.h5_pris = dfHist.h5_pris.astype('float')


In [ ]:
dfHist.h1_plac = dfHist.h1_plac.str.replace('r','',regex=False)
dfHist.h2_plac = dfHist.h2_plac.str.replace('r','',regex=False)
dfHist.h3_plac = dfHist.h3_plac.str.replace('r','',regex=False)
dfHist.h4_plac = dfHist.h4_plac.str.replace('r','',regex=False)
dfHist.h5_plac = dfHist.h5_plac.str.replace('r','',regex=False)
dfHist.h1_plac = dfHist.h1_plac.str.replace('.0','',regex=False)
dfHist.h2_plac = dfHist.h2_plac.str.replace('.0','',regex=False)
dfHist.h3_plac = dfHist.h3_plac.str.replace('.0','',regex=False)
dfHist.h4_plac = dfHist.h4_plac.str.replace('.0','',regex=False)
dfHist.h5_plac = dfHist.h5_plac.str.replace('.0','',regex=False)
# df.plac = df.plac.str.replace('.0','',regex=False)


In [ ]:
dfHist.info()

## lägg in ny history i stora databasen

In [ ]:
# dfHist=pd.read_csv('mer_history.csv')
# dfHist.info()

In [ ]:
hist_ix = dfHist.set_index(['datum','avd','häst']).sort_index()

In [ ]:
mer_komplett=pd.read_csv('mer_komplett.csv')
mer_komplett.info()


In [ ]:
mer_kompl_ix=mer_komplett.set_index(['datum','avd','häst']).sort_index()

In [ ]:
mer_kompl_ix[[ 'h1_dat','h1_bana','h1_kusk','h1_plac','h1_dist','h1_spår','h1_kmtid','h1_odds',	'h1_pris',
            'h2_dat','h2_bana','h2_kusk','h2_plac',	'h2_dist','h2_spår','h2_kmtid',	'h2_odds','h2_pris',
            'h3_dat','h3_bana',	'h3_kusk','h3_plac','h3_dist','h3_spår','h3_kmtid',	'h3_odds','h3_pris',
            'h4_dat','h4_bana',	'h4_kusk','h4_plac','h4_dist','h4_spår','h4_kmtid',	'h4_odds','h4_pris',                                        'h5_dat','h5_bana',	'h5_kusk','h5_plac','h5_dist','h5_spår','h5_kmtid',	'h5_odds','h5_pris',        
	]]

In [ ]:
cols=[ 'h1_bana', 'h1_kusk', 'h1_plac', 'h1_dist','h1_spår','h1_kmtid','h1_odds','h1_pris',
    'h2_dat', 'h2_bana', 'h2_kusk', 'h2_plac', 'h2_dist','h2_spår','h2_kmtid','h2_odds','h2_pris',
    'h3_dat', 'h3_bana', 'h3_kusk', 'h3_plac', 'h3_dist','h3_spår','h3_kmtid','h3_odds','h3_pris',
    'h4_dat', 'h4_bana', 'h4_kusk', 'h4_plac', 'h4_dist','h4_spår','h4_kmtid','h4_odds','h4_pris',
    'h5_dat', 'h5_bana', 'h5_kusk', 'h5_plac', 'h5_dist','h5_spår','h5_kmtid','h5_odds','h5_pris','h1_dat',
    ]

spara_mer=mer_kompl_ix
spara_hist=hist_ix
cnt=0
for i in hist_ix.index:
    for col in cols:
        # print(mer_kompl_ix[col].loc[i], df_hist_ix[col].loc[i])
        mer_kompl_ix[col].loc[i] = hist_ix[col].loc[i]  
        # print(mer_kompl_ix[col].loc[i], df_hist_ix[col].loc[i])
    cnt+=1        
    if cnt%1000==0: 
        print(cnt)


In [ ]:
mer_kompl_ix.reset_index(inplace=True)

In [ ]:
print(mer_kompl_ix.shape)
# mer_kompl_ix.drop('index',axis=1,inplace=True)
# mer_kompl_ix.drop('level_0',axis=1,inplace=True)
mer_kompl_ix.info()

In [ ]:
mer_kompl_ix.to_csv('mer_komplett.csv',index=False)

In [ ]:
# mer_kompl_ix = pd.read_csv('mer_komplett.csv')
#PRINS C.K. saknas i lista. 2020-06-13 BODEN avd=7
print(mer_kompl_ix.shape)
print(mer_kompl_ix[(mer_kompl_ix.datum=='2019-08-31') &	(mer_kompl_ix.häst=='JULA DOWNTON')][['h1_dat','h1_bana', 'h1_kusk', 'h1_plac', 'h1_dist','h1_spår','h1_kmtid','h1_odds','h1_pris']])
print()

print(mer_kompl_ix[(mer_kompl_ix.datum=='2019-08-31') &	(mer_kompl_ix.häst=='JULA DOWNTON')][['h2_dat', 'h2_bana', 'h2_kusk', 'h2_plac', 'h2_dist','h2_spår','h2_kmtid','h2_odds','h2_pris']])
print()
print(mer_kompl_ix[(mer_kompl_ix.datum=='2019-08-31') &	(mer_kompl_ix.häst=='JULA DOWNTON')][['h3_dat', 'h3_bana', 'h3_kusk', 'h3_plac', 'h3_dist','h3_spår','h3_kmtid','h3_odds','h3_pris']])
print(mer_kompl_ix[(mer_kompl_ix.datum=='2019-08-31') &	(mer_kompl_ix.häst=='JULA DOWNTON')][['h4_dat', 'h4_bana', 'h4_kusk', 'h4_plac', 'h4_dist','h4_spår','h4_kmtid','h4_odds','h4_pris']])
print(mer_kompl_ix[(mer_kompl_ix.datum=='2019-08-31') &	(mer_kompl_ix.häst=='JULA DOWNTON')][['h5_dat', 'h5_bana', 'h5_kusk', 'h5_plac', 'h5_dist','h5_spår','h5_kmtid','h5_odds','h5_pris','h1_dat']]=='')


   

# Testa lite

In [ ]:
# Testa lite
stor_df = pd.DataFrame(columns=['a','b','c','d','e'])
stor_df.loc[stor_df.shape[0]] = [11,21,31,'d1','e1']
stor_df.loc[stor_df.shape[0]] = [12,22,32,'d2','e2']
stor_df.loc[stor_df.shape[0]] = [13,23,33,'d3','e3']
stor_df.loc[stor_df.shape[0]] = [14,24,34,'d4','e4']
stor_df.loc[stor_df.shape[0]] = [14,25,35,'d5','e5']
stor_df=stor_df.set_index(['a','b'])
print(stor_df)
liten_df = pd.DataFrame(columns=['a','b','c','e'])
liten_df.loc[liten_df.shape[0]] = [11,21,311,'e11']
liten_df.loc[liten_df.shape[0]] = [12,22,321,'e21']
liten_df.loc[liten_df.shape[0]] = [14,24,351,'e41']
liten_df.loc[liten_df.shape[0]] = [14,25,352,'e51']

liten_df=liten_df.set_index(['a','b']).sort_index()
liten_df

In [ ]:
liten=liten_df
stor=stor_df
for s in liten.index:
    print(s,liten.loc[s].c)
    print('stor',stor.c.loc[s])
    print('liten',liten.loc[s].c)
    stor['c'].loc[s] = liten.loc[s].c
    stor.e.loc[s] = liten.loc[s].e

stor_df